# Geo analytics AAA - Intentionally Blank

Census_tracts_borders = https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Census-Tracts-2010/5jrd-6zik
https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2016&layergroup=Census+Tracts

In [ ]:
import json

import pandas as pd
import numpy as np

import folium
import h3
import matplotlib.pyplot as plt
import geopandas as gpd
import branca.colormap as cm
import shapely
from shapely.geometry import shape
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe+notebook+plotly_mimetype'


In [ ]:
taxi_df = pd.read_parquet('data/prepared/taxi_data_prepared.gzip')
taxi_df['pickup_centroid_location'] = gpd.GeoSeries.from_wkt(taxi_df['pickup_centroid_location'])
taxi_df['dropoff_centroid_location'] = gpd.GeoSeries.from_wkt(taxi_df['dropoff_centroid_location'])
census_tract_borders = gpd.read_file('data/chicago_census_tract_borders.zip')

In [ ]:
taxi_df


In [ ]:
census_tract_borders = census_tract_borders.drop(census_tract_borders.columns.difference(['GEOID', 'geometry']), axis=1)

In [ ]:
unique_census_tract_id = np.append(taxi_df['pickup_census_tract'].unique(), taxi_df['dropoff_census_tract'].unique()).astype('str')
census_tract_borders= census_tract_borders[census_tract_borders['GEOID'].isin(unique_census_tract_id)].reset_index(drop=True)
census_tract_borders

In [ ]:
def plotByFeatureStatic(dataframe, location='pickup', feature='all', aggregation='sum', missingCensusTract=False):
    """ Plot a feature of a dataframe on a map.

    Parameters
    ----------

    dataframe :  (pandas.DataFrame) 
        The dataframe to plot.
    location : (str) 
        The location column of the dataframe. Can be either 'pickup' or 'dropoff'. Default is 'pickup'.
    feature : (str) 
        The feature to aggregate. If 'all', all features are aggregated. Default is 'all'
    aggregation : (str)  
        The aggregation function to use. Can be either 'mean', 'median', 'sum', 'count', 'min', 'max'. Default is 'sum'.
    containMissingCensusTract : (bool)
        If True, census tracts with no data are included in the plot. Default is False.

    Returns
    ----------

    dataframe_grouped : (geopandas.GeoDataFrame) 
        The geodataframe grouped by the location column and the feature column. Contains always a geometry column and trip_count column.
    """
    dataframe_grouped = dataframe.copy()
    dataframe_grouped.dropna(inplace = True)
    if feature == 'all':
        features = dataframe_grouped.columns.difference(['pickup_census_tract', 'dropoff_census_tract', 'pickup_centroid_location', 'dropoff_centroid_location', 'trip_start_timestamp', 'trip_end_timestamp', 'taxi_id']).tolist()
    else:
        features = [feature]
    
    if location == 'pickup':
        features.append('pickup_census_tract')
        features.append('pickup_centroid_location')
        dataframe_grouped = dataframe_grouped.drop(columns=dataframe_grouped.columns.difference(features))
    elif location == 'dropoff':
        features.append('dropoff_census_tract')
        features.append('dropoff_centroid_location')
        dataframe_grouped = dataframe_grouped.drop(columns=dataframe_grouped.columns.difference(features))
    else:
        raise ValueError("Location must be either 'pickup' or 'dropoff'.")
    dataframe_grouped['trip_count'] = dataframe_grouped[features[0]]
    dataframe_grouped = dataframe_grouped.groupby([location + '_census_tract', location + '_centroid_location']).agg(lambda column: column.agg('count') if column.name == 'trip_count' else column.agg(aggregation)).reset_index()
    dataframe_grouped = dataframe_grouped.rename(columns={location + '_census_tract': 'GEOID'})
    dataframe_grouped['GEOID'] = dataframe_grouped['GEOID'].astype('str')
    if missingCensusTract == True:
        dataframe_grouped = dataframe_grouped.merge(census_tract_borders, on='GEOID', how='right')
        dataframe_grouped = gpd.GeoDataFrame(dataframe_grouped)
        dataframe_grouped[location + '_centroid_location'] = dataframe_grouped['geometry'].to_crs('+proj=cea').centroid.to_crs('EPSG:4326')
    else:
        dataframe_grouped = dataframe_grouped.merge(census_tract_borders, on='GEOID', how='left')
        dataframe_grouped = gpd.GeoDataFrame(dataframe_grouped)
    return dataframe_grouped


In [ ]:
geo_df_trip_seconds = plotByFeatureStatic(taxi_df, missingCensusTract = True, feature='all', aggregation= "sum")

In [ ]:
geo_df_trip_seconds[geo_df_trip_seconds["trip_total"] == 0]

In [ ]:
loc = 'Trip count by census tract (logarithmic)'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m = folium.Map(location=(41.8,-87.723177), zoom_start=11)

m.get_root().html.add_child(folium.Element(title_html))
geo_df_trip_seconds.explore(column=np.log10(geo_df_trip_seconds['trip_count']), tooltip=True, cmap="viridis", m = m)

In [ ]:
# Placeholder
loc = 'Trip total by census tract (logarithmic)'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m = folium.Map(location=(41.8,-87.723177), zoom_start=11)

m.get_root().html.add_child(folium.Element(title_html))
geo_df_trip_seconds.explore(column=np.log10(geo_df_trip_seconds['trip_total']), tooltip=True, cmap="viridis", m = m)

In [ ]:
fig, axd = plt.subplot_mosaic([['trip_total', 'trip_seconds'],
                               ['trip_miles', 'idle_seconds'],
                               ['trip_count', 'trip_count']],
                              figsize=(12,20), layout="constrained")
geo_df_trip_seconds.plot(column=np.log10(geo_df_trip_seconds['trip_total']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_total'], legend_kwds={"label": "Average Revenue (in dollar)", "orientation": "horizontal"})
geo_df_trip_seconds.plot(column=np.log10(geo_df_trip_seconds['trip_seconds']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_seconds'], legend_kwds={"label": "Average trip time in seconds", "orientation": "horizontal"})
geo_df_trip_seconds.plot(column=np.log10(geo_df_trip_seconds['trip_miles']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_miles'], legend_kwds={"label": "Average trip miles in seconds", "orientation": "horizontal"})
geo_df_trip_seconds[geo_df_trip_seconds['idle_seconds'] > 0].plot(column=np.log10(geo_df_trip_seconds[geo_df_trip_seconds['idle_seconds'] > 0]['idle_seconds']),  cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['idle_seconds'], legend_kwds={"label": "Average idle time in seconds", "orientation": "horizontal"})
geo_df_trip_seconds.plot(column=np.log10(geo_df_trip_seconds['trip_count']), cmap="viridis", legend=True,missing_kwds={'color': 'lightgrey'}, ax=axd['trip_count'], legend_kwds={"label": "Trips per district", "orientation": "horizontal"})

In [ ]:
geo_df_trip_seconds["trip_count"].sum()

In [ ]:
# Create docstring for the following function
def plotH3_HexagonMap(dataframe, location='pickup', feature='all', aggregation='sum', missingCensusTract=False, hexRes = 10):
    """ Plot a feature of a dataframe on a map.

    Parameters
    ----------

    dataframe :  (pandas.DataFrame) 
        The dataframe to plot.
    location : (str) 
        The location column of the dataframe. Can be either 'pickup' or 'dropoff'. Default is 'pickup'.
    feature : (str) 
        The feature to aggregate. If 'all', all features are aggregated. Default is 'all'
    aggregation : (str)  
        The aggregation function to use. Can be either 'mean', 'median', 'sum', 'count', 'min', 'max'. Default is 'sum'.
    containMissingCensusTract : (bool)
        If True, census tracts with no data are included in the plot. Default is False.
    hexRes : (int)
        H3 hexagon resolution size. Default is 10.
    Returns
    ----------

    taxi_df_geo_grouped : (geopandas.GeoDataFrame) 
        The geodataframe grouped by the location column and the feature column. Contains always a geometry column and trip_count column.
    """
    taxi_df_geo = plotByFeatureStatic(dataframe, location= location,feature = feature, aggregation = aggregation, missingCensusTract = missingCensusTract)
    # geometry to h3 index
    taxi_df_geo['h3_index'] = taxi_df_geo.apply(lambda row: h3.geo_to_h3(row[location + '_centroid_location'].y, row[location + '_centroid_location'].x, hexRes), axis=1)

    geojson = []
    geometries = []
    indexes = []

    for geometry in taxi_df_geo['geometry']:
        geojson.append(shapely.to_geojson(geometry))

    for geometry in geojson:
        obj = json.loads(geometry)
        h3_indexes = h3.polyfill(obj, hexRes ,True)
        for index in h3_indexes:
            geometries.append(shape({"type": "Polygon",
                    "coordinates": [h3.h3_to_geo_boundary(index, geo_json=True)],
                    "properties": ""
                    }))
            indexes.append(index)   
    taxi_df_geo.drop(columns= ['geometry', 'pickup_centroid_location', 'GEOID'], inplace = True)
    df_h3_polyfilled = pd.DataFrame({'h3_index': indexes})
    taxi_df_geo_grouped = taxi_df_geo.groupby('h3_index').agg(aggregation).reset_index()
    taxi_df_geo_grouped = taxi_df_geo_grouped.merge(df_h3_polyfilled, on='h3_index', how='outer')
    taxi_df_geo_grouped['geometry'] = taxi_df_geo_grouped.apply(lambda row: shape({"type": "Polygon",
                                           "coordinates": [h3.h3_to_geo_boundary(row["h3_index"], geo_json=True)],
                                           "properties": ""
                                           }), axis=1)
    taxi_df_geo_grouped = gpd.GeoDataFrame(taxi_df_geo_grouped, crs='EPSG:4326', geometry='geometry')
    return taxi_df_geo_grouped


In [ ]:
taxi_df_geo_h3 = plotH3_HexagonMap(taxi_df, location='pickup', feature='all', aggregation='sum', missingCensusTract=True, hexRes= 8).explore(column='trip_count', tooltip=True, cmap="viridis")
taxi_df_geo_h3

In [ ]:
def plotByFeatureTime(dataframe, location='pickup', feature='all', aggregation='sum', time_interval='hour', start_time = True):
    """ Plot a feature of a dataframe on a map.

    Parameters
    ----------

    dataframe :  (pandas.DataFrame) 
        The dataframe to plot.
    location : (str) 
        The location column of the dataframe. Can be either 'pickup' or 'dropoff'. Default is 'pickup'.
    feature : (str) 
        The feature to aggregate. If 'all', all features are aggregated. Default is 'all'
    aggregation : (str)  
        The aggregation function to use. Can be either 'mean', 'median', 'sum', 'count', 'min', 'max'. Default is 'sum'.
    containMissingCensusTract : (bool)
        If True, census tracts with no data are included in the plot. Default is False.

    Returns
    ----------

    dataframe_grouped : (geopandas.GeoDataFrame) 
        The geodataframe grouped by the location column and the feature column. Contains always a geometry column and trip_count column.
    """
    dataframe_grouped = dataframe.copy()
    dataframe_grouped.dropna(inplace = True)
    if feature == 'all':
        features = dataframe_grouped.columns.difference(['pickup_census_tract', 'dropoff_census_tract', 'pickup_centroid_location', 'dropoff_centroid_location']).tolist()
    else:
        features = [feature]
    
    if location == 'pickup':
        features.append('pickup_census_tract')
        features.append('pickup_centroid_location')
        dataframe_grouped = dataframe_grouped.drop(columns=['dropoff_census_tract', 'dropoff_centroid_location'])
    elif location == 'dropoff':
        features.append('dropoff_census_tract')
        features.append('dropoff_centroid_location')
        dataframe_grouped = dataframe_grouped.drop(columns=['pickup_census_tract', 'pickup_centroid_location'])
    else:
        raise ValueError("Location must be either 'pickup' or 'dropoff'.")

    if start_time == True:
        dataframe_grouped.rename(columns={'trip_start_timestamp': 'timestamp'}, inplace=True)
        dataframe_grouped.drop(columns=['trip_end_timestamp'], inplace=True)
    else:
        dataframe_grouped.rename(columns={'trip_end_timestamp': 'timestamp'}, inplace=True)
        dataframe_grouped.drop(columns=['trip_start_timestamp'], inplace=True)
    features[0]='timestamp'
    if time_interval == 'hour':
        dataframe_grouped["hour"] = dataframe_grouped['timestamp'].dt.hour
        dataframe_grouped.rename(columns={'timestamp': 'trip_count'}, inplace=True)
        dataframe_grouped = dataframe_grouped.groupby([location + '_census_tract', location + '_centroid_location', 'hour']).agg(lambda column: column.agg('count') if column.name == 'trip_count' else column.agg(aggregation)).reset_index()
    elif time_interval == 'month':
        dataframe_grouped["month"] = dataframe_grouped['timestamp'].dt.month
        dataframe_grouped.rename(columns={'timestamp': 'trip_count'}, inplace=True)
        dataframe_grouped = dataframe_grouped.groupby([location + '_census_tract', location + '_centroid_location', 'month']).agg(lambda column: column.agg('count') if column.name == 'trip_count' else column.agg(aggregation)).reset_index()
    elif time_interval == 'weekday':
        dataframe_grouped["weekday"] = dataframe_grouped['timestamp'].dt.weekday
        dataframe_grouped.rename(columns={'timestamp': 'trip_count'}, inplace=True)
        dataframe_grouped = dataframe_grouped.groupby([location + '_census_tract', location + '_centroid_location', 'weekday']).agg(lambda column: column.agg('count') if column.name == 'trip_count' else column.agg(aggregation)).reset_index()
    dataframe_grouped["trip_count"] =dataframe_grouped["trip_count"].astype("int32")
    dataframe_grouped = dataframe_grouped.rename(columns={location + '_census_tract': 'GEOID'})
    dataframe_grouped['GEOID'] = dataframe_grouped['GEOID'].astype('str')
    dataframe_grouped = dataframe_grouped.merge(census_tract_borders, on='GEOID', how='left')
    dataframe_grouped = gpd.GeoDataFrame(dataframe_grouped)
    dataframe_grouped.sort_values(by=[time_interval], inplace=True)
    return dataframe_grouped

In [ ]:
interactivePlotPickup = plotByFeatureTime(taxi_df, location='pickup', feature='all', aggregation='sum', time_interval='hour', start_time = True)

In [ ]:
interactivePlotDropoff = plotByFeatureTime(taxi_df, location='dropoff', feature='all', aggregation='sum', time_interval='hour', start_time = True)

In [ ]:
viridis = px.colors.sequential.Viridis
fig = px.choropleth_mapbox(interactivePlotPickup,
                        geojson=interactivePlotPickup[['GEOID', 'geometry']].drop_duplicates(subset=['GEOID']), 
                        locations=interactivePlotPickup.GEOID,
                        featureidkey="properties.GEOID",
                        color='trip_count',
                        animation_frame='hour',
                        mapbox_style='open-street-map', 
                        zoom=8,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 300000],
                        center={'lat': 41.881832, 'lon': -87.623177},
                        labels={'trip_count':'trip_count'},
                        width=800,
                        height=800)
fig.show()
fig = px.choropleth_mapbox(interactivePlotDropoff,
                        geojson=interactivePlotDropoff[['GEOID', 'geometry']].drop_duplicates(subset=['GEOID']), 
                        locations=interactivePlotDropoff.GEOID,
                        featureidkey="properties.GEOID",
                        color='trip_count',
                        animation_frame='hour',
                        mapbox_style='open-street-map', 
                        zoom=8,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 300000],
                        center={'lat': 41.881832, 'lon': -87.623177},
                        labels={'trip_count':'trip_count'},
                        width=800,
                        height=800)
fig.show()

In [ ]:
# Create docstring for the following function
def plotH3_HexagonMapInteractive(dataframe, location='pickup', feature='all', aggregation='sum', missingCensusTract=False, hexRes = 10, time_interval='hour'):
    """ Plot a feature of a dataframe on a map.

    Parameters
    ----------

    dataframe :  (pandas.DataFrame) 
        The dataframe to plot.
    location : (str) 
        The location column of the dataframe. Can be either 'pickup' or 'dropoff'. Default is 'pickup'.
    feature : (str) 
        The feature to aggregate. If 'all', all features are aggregated. Default is 'all'
    aggregation : (str)  
        The aggregation function to use. Can be either 'mean', 'median', 'sum', 'count', 'min', 'max'. Default is 'sum'.
    containMissingCensusTract : (bool)
        If True, census tracts with no data are included in the plot. Default is False.
    hexRes : (int)
        H3 hexagon resolution size. Default is 10.
    Returns
    ----------

    taxi_df_geo_grouped : (geopandas.GeoDataFrame) 
        The geodataframe grouped by the location column and the feature column. Contains always a geometry column and trip_count column.
    """
    taxi_df_geo = plotByFeatureTime(dataframe, location= location,feature = feature, aggregation = aggregation, time_interval=time_interval)
    # geometry to h3 index
    taxi_df_geo['h3_index'] = taxi_df_geo.apply(lambda row: h3.geo_to_h3(row[location + '_centroid_location'].y, row[location + '_centroid_location'].x, hexRes), axis=1)

    geojson = []
    geometries = []
    indexes = []

    for geometry in taxi_df_geo['geometry']:
        geojson.append(shapely.to_geojson(geometry))

    for geometry in geojson:
        obj = json.loads(geometry)
        h3_indexes = h3.polyfill(obj, hexRes ,True)
        for index in h3_indexes:
            geometries.append(shape({"type": "Polygon",
                    "coordinates": [h3.h3_to_geo_boundary(index, geo_json=True)],
                    "properties": ""
                    }))
            indexes.append(index)   
    taxi_df_geo.drop(columns= ['geometry', 'pickup_centroid_location', 'GEOID'], inplace = True)
    df_h3_polyfilled = pd.DataFrame({'h3_index': indexes})
    taxi_df_geo_grouped = taxi_df_geo.groupby(['h3_index', time_interval]).agg(aggregation).reset_index()
    taxi_df_geo_grouped = taxi_df_geo_grouped.merge(df_h3_polyfilled, on='h3_index', how='outer')
    taxi_df_geo_grouped['geometry'] = taxi_df_geo_grouped.apply(lambda row: shape({"type": "Polygon",
                                           "coordinates": [h3.h3_to_geo_boundary(row["h3_index"], geo_json=True)],
                                           "properties": ""
                                           }), axis=1)
    taxi_df_geo_grouped = gpd.GeoDataFrame(taxi_df_geo_grouped, crs='EPSG:4326', geometry='geometry')
    taxi_df_geo_grouped.sort_values(by=[time_interval], inplace=True)
    return taxi_df_geo_grouped


In [ ]:
interactivePlotPickuphex = plotH3_HexagonMapInteractive(taxi_df, location='pickup', feature='all', aggregation='sum', time_interval='hour', hexRes=8)

In [ ]:
interactivePlotPickuphex

In [ ]:
fig = px.choropleth_mapbox(interactivePlotPickuphex,
                        geojson=interactivePlotPickuphex[['h3_index', 'geometry']].drop_duplicates(subset=['h3_index']), 
                        locations=interactivePlotPickuphex.h3_index,
                        featureidkey="properties.h3_index",
                        color='trip_count',
                        animation_frame='hour',
                        mapbox_style='open-street-map', 
                        zoom=9,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 300000],
                        center={'lat': 41.881832, 'lon': -87.623177},
                        labels={'trip_count':'trip_count'},
                        width=800,
                        height=800)
fig.show()